In [366]:
import numpy as np
import pandas as pd
import nba_api
from sklearn import linear_model

In [77]:
from nba_api.stats.endpoints import leaguegamefinder
from nba_api.stats.static import teams


from nba_api.stats.endpoints import teamdashboardbylastngames as teamdashbyn


In [11]:
nba_teams = teams.get_teams()
knicks = [team for team in nba_teams if team['abbreviation'] == 'NYK'][0]
knicks_id = knicks['id']

In [136]:
gamefinder = leaguegamefinder.LeagueGameFinder(league_id_nullable='00', season_nullable='2018-19', season_type_nullable='Regular Season')

In [137]:
def makeOneHot(id):
    onehot = np.zeros(30)
    onehot[id] = 1
    return onehot

In [162]:
df = gamefinder.get_data_frames()[0]
base_team_id = df['TEAM_ID'].unique().min()
df['TEAM_ID'] = df['TEAM_ID'] - base_team_id
ab_to_id = df.loc[:,['TEAM_ID', 'TEAM_ABBREVIATION']].set_index('TEAM_ABBREVIATION').to_dict()
ab_to_id = ab_to_id['TEAM_ID']
ab_to_id

{'MEM': 26,
 'DET': 28,
 'NYK': 15,
 'PHI': 18,
 'GSW': 7,
 'CHI': 4,
 'IND': 17,
 'LAC': 9,
 'CHA': 29,
 'UTA': 25,
 'DAL': 5,
 'POR': 20,
 'DEN': 6,
 'ORL': 16,
 'ATL': 0,
 'BKN': 14,
 'OKC': 23,
 'SAC': 21,
 'MIL': 12,
 'SAS': 22,
 'MIN': 13,
 'MIA': 11,
 'HOU': 8,
 'LAL': 10,
 'BOS': 1,
 'PHX': 19,
 'WAS': 27,
 'CLE': 2,
 'TOR': 24,
 'NOP': 3}

In [180]:
def matchupToRow(matchup):
    if 'vs.' in matchup:
        home = matchup[0:3]
        away = matchup[-3:]
        homeid = ab_to_id[home]
        awayid = ab_to_id[away]
        homeoh = makeOneHot(homeid)
        awayoh = makeOneHot(awayid)
        return np.hstack((homeoh, awayoh))

In [184]:
df = df.sort_values(by='GAME_ID')
matchupToRow(df.iloc[0]['MATCHUP'])

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
2456,22018,1,BOS,Boston Celtics,0021800001,2018-10-16,BOS vs. PHI,W,240,105,...,0.714,12,43,55,21,7,5,14,20,18.0
2458,22018,18,PHI,Philadelphia 76ers,0021800001,2018-10-16,PHI @ BOS,L,239,87,...,0.609,6,41,47,18,8,5,16,20,-18.0
2459,22018,7,GSW,Golden State Warriors,0021800002,2018-10-16,GSW vs. OKC,W,241,108,...,0.944,17,41,58,28,7,7,21,29,8.0
2457,22018,23,OKC,Oklahoma City Thunder,0021800002,2018-10-16,OKC @ GSW,L,240,100,...,0.649,16,29,45,21,12,6,14,21,-8.0
2440,22018,29,CHA,Charlotte Hornets,0021800003,2018-10-17,CHA vs. MIL,L,241,112,...,0.636,9,32,41,21,8,9,11,19,-1.0
2452,22018,12,MIL,Milwaukee Bucks,0021800003,2018-10-17,MIL @ CHA,W,240,113,...,0.750,11,46,57,26,5,4,21,25,1.0
2446,22018,28,DET,Detroit Pistons,0021800004,2018-10-17,DET vs. BKN,W,240,103,...,0.864,14,32,46,21,5,5,14,20,3.0
2437,22018,14,BKN,Brooklyn Nets,0021800004,2018-10-17,BKN @ DET,L,240,100,...,0.682,5,34,39,28,9,5,17,23,-3.0
2450,22018,17,IND,Indiana Pacers,0021800005,2018-10-17,IND vs. MEM,W,240,111,...,0.538,13,44,57,29,2,7,20,24,28.0
2434,22018,26,MEM,Memphis Grizzlies,0021800005,2018-10-17,MEM @ IND,L,240,83,...,0.821,7,21,28,16,11,3,7,18,-28.0


In [260]:
def flattenAway(home, away):
    home.assign(AWAY_PTS=np.zeros(home.shape[0]))
    home.assign(AWAY_TEAM_ABBREVIATION=np.zeros(home.shape[0]))
    for i, row in home.iterrows():
        game_id = row['GAME_ID']
        otherRow = away[away['GAME_ID'] == game_id]
        home.at[i, "AWAY_PTS"] = otherRow['PTS'].iloc[0]
        home.at[i, "AWAY_TEAM_ABBREVIATION"] = otherRow["TEAM_ABBREVIATION"].iloc[0]
    return home

In [264]:
home = df[df.MATCHUP.str.contains('vs.')]
away = df[df.MATCHUP.str.contains('@')]


home = flattenAway(home,away)
home

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS,AWAY_PTS,AWAY_TEAM_ABBREVIATION
2456,22018,1,BOS,Boston Celtics,0021800001,2018-10-16,BOS vs. PHI,W,240,105,...,43,55,21,7,5,14,20,18.0,87.0,PHI
2459,22018,7,GSW,Golden State Warriors,0021800002,2018-10-16,GSW vs. OKC,W,241,108,...,41,58,28,7,7,21,29,8.0,100.0,OKC
2440,22018,29,CHA,Charlotte Hornets,0021800003,2018-10-17,CHA vs. MIL,L,241,112,...,32,41,21,8,9,11,19,-1.0,113.0,MIL
2446,22018,28,DET,Detroit Pistons,0021800004,2018-10-17,DET vs. BKN,W,240,103,...,32,46,21,5,5,14,20,3.0,100.0,BKN
2450,22018,17,IND,Indiana Pacers,0021800005,2018-10-17,IND vs. MEM,W,240,111,...,44,57,29,2,7,20,24,28.0,83.0,MEM
2453,22018,16,ORL,Orlando Magic,0021800006,2018-10-17,ORL vs. MIA,W,240,104,...,37,51,20,7,7,12,25,3.0,101.0,MIA
2435,22018,15,NYK,New York Knicks,0021800007,2018-10-17,NYK vs. ATL,W,240,126,...,36,46,21,12,6,15,23,19.0,107.0,ATL
2448,22018,24,TOR,Toronto Raptors,0021800008,2018-10-17,TOR vs. CLE,W,239,116,...,35,43,21,6,7,9,29,12.0,104.0,CLE
2444,22018,8,HOU,Houston Rockets,0021800009,2018-10-17,HOU vs. NOP,L,240,112,...,29,37,21,8,7,11,22,-19.0,131.0,NOP
2454,22018,22,SAS,San Antonio Spurs,0021800010,2018-10-17,SAS vs. MIN,W,241,112,...,33,52,22,3,4,12,22,4.0,108.0,MIN


In [318]:
def mapToNumpy(df):
    x = None
    for i, row in df.iterrows():
        currow = matchupToRow(row['MATCHUP'])
        currow = np.append(currow, row['PTS'] - row['AWAY_PTS']);
        if currow is None:
            print('fukc')
        if x is None:
            x = currow
        else:
            x = np.vstack((x, currow))
    return x

In [344]:
xy = mapToNumpy(home)


In [390]:
minloss = 1000000000.00
best_model = None

for i in range(100):
    np.random.shuffle(xy)
    x = xy[:,0:-1]
    y = xy[:,-1]
    trainEndIndex = int((x.shape[0] * 7) / 10)
    xtrain = x[0:trainEndIndex,:]
    ytrain = y[0:trainEndIndex]

    xval = x[trainEndIndex:,:]
    yval = y[trainEndIndex:]

    reg = linear_model.Ridge(alpha=.5)
    reg.fit(xtrain,ytrain)

    w = np.dot(xval, reg.coef_)
    avg_loss = np.linalg.norm(w - yval) / w.shape[0]
    if avg_loss < minloss:
        minloss = avg_loss
        best_model = reg.coef_
minloss

0.6486308914651185

Ridge(alpha=0.5, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=False, random_state=None, solver='auto', tol=0.001)

array([ -8.34113041,   6.75859306, -11.60202202,  -0.80380463,
       -12.0383004 ,   2.40390797,   9.32472269,   4.31187774,
         2.66054338,  -0.60888886,  -2.83104135,  -4.44210908,
         9.37756106,   2.18136184,  -1.89292318,  -9.73908124,
        -5.44938047,   5.4849557 ,   5.15435623,  -8.77554728,
         5.7093897 ,  -1.03546269,   4.34459956,   3.58689113,
         4.7579173 ,   3.63267529,  -3.31942105,   0.8101906 ,
        -1.7476731 ,   2.1272425 ,   5.52383371,  -4.14211148,
         8.26792877,  -2.4286985 ,   3.76169364,   1.53222803,
         0.5686278 , -10.7746539 ,  -2.44274409,  -1.94227685,
        -0.96404376,  -1.17214947,  -9.03835144,   2.35635848,
         0.18572996,   6.16374936,  -1.88075982,  -1.45604966,
        -0.88749255,   8.19769504,   0.44496441,   1.40629565,
         1.67424857,  -5.48266482,  -5.71205898,  -4.46147432,
         1.91649631,   6.96405703,   1.27547745,   2.54614541])

0.7141827403230748